In [1]:
!nvidia-smi

Sat Aug 14 17:37:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -U torch==1.4 torchvision==0.5 -f https://download.pytorch.org/whl/cu101/torch_stable.html

import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/xingyizhou/CenterNet.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone
  !git clone -q --depth 1 $git_repo_url
  # fix DCNv2
  !cd {project_name}/src/lib/models/networks && rm -rf DCNv2 && git clone https://github.com/CharlesShang/DCNv2.git && cd DCNv2 && ./make.sh
  # dependencies
  !cd $project_name && pip install -q -r requirements.txt

import sys
sys.path.insert(0, join(project_name, 'src/lib'))
sys.path.append(join(project_name, 'src'))
# following 2 lines needed to avoid later import error
sys.path.append(join(project_name, 'src/lib/models/networks/DCNv2'))
from dcn_v2 import DCN

import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

from IPython.display import clear_output

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 753.4 MB 7.0 kB/s 
     |████████████████████████████████| 4.0 MB 59.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.
Cloning into 'DCNv2'...
remote: Enumerating objects: 214, done.
remote: Total 214 (delta 0), reused 0 (delta 0), pack-reused 214
Receiving objects: 100% (214/214), 1.42 MiB | 1.66 MiB/s,

In [3]:
model_name = 'multi_pose_dla_3x.pth'
if not exists(model_name):
  !pip install -q gdown
  !gdown 'https://drive.google.com/uc?id=1PO1Ax_GDtjiemEmDVD7oPWwqQkUu28PI'

Downloading...
From: https://drive.google.com/uc?id=1PO1Ax_GDtjiemEmDVD7oPWwqQkUu28PI
To: /content/multi_pose_dla_3x.pth
82.7MB [00:01, 74.1MB/s]


In [4]:
# !mv multi_pose_dla_3x.pth CenterNet/models/

In [5]:
#動画ファイルを読み込むため、Google Driveをマウントする
from google.colab import drive
drive.mount('/content/drive') #変更不要

Mounted at /content/drive


In [6]:
#読み込みたい動画のPathを指定する(要変更)
VIDEO_PATH = "/content/drive/Shareddrives/Ex_3days_サマーインターン事前課題提出先/福田晃大/demo_walking.mov"

In [7]:
#変更不要
CENTERNET_LIB_PATH = 'CenterNet/src/lib'
MODEL_PATH = 'multi_pose_dla_3x.pth'
# !cd CenterNet/src
# !cd ../../
!ls

CenterNet  drive  multi_pose_dla_3x.pth  sample_data


In [8]:
import glob as glob
import sys
sys.path.insert(0, CENTERNET_LIB_PATH)
import warnings
warnings.filterwarnings('ignore')

import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

# import pickle
# from PIL import Image, ImageFont, ImageDraw
# import colorsys
# from pylab import rcParams

from detectors.detector_factory import detector_factory
from opts import opts

NMS not imported! If you need it, do 
 cd $CenterNet_ROOT/src/lib/external 
 make
NMS not imported! If you need it, do 
 cd $CenterNet_ROOT/src/lib/external 
 make


In [9]:
TASK = 'multi_pose' # or 'multi_pose' for human pose estimation
opt = opts().init('{} --load_model {}'.format(TASK, MODEL_PATH).split(' '))
detector = detector_factory[opt.task](opt)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  CenterNet/src/lib/../../exp/multi_pose/default
heads {'hm': 1, 'wh': 2, 'hps': 34, 'reg': 2, 'hm_hp': 17, 'hp_offset': 2}
Creating model...


Downloading: "http://dl.yf.io/dla/models/imagenet/dla34-ba72cf86.pth" to /root/.cache/torch/checkpoints/dla34-ba72cf86.pth


  0%|          | 0.00/60.3M [00:00<?, ?B/s]

loaded multi_pose_dla_3x.pth, epoch 320


In [17]:
colors_hp = [(255, 0, 255), (255, 0, 0), (0, 0, 255), 
        (255, 0, 0), (0, 0, 255), (255, 0, 0), (0, 0, 255),
        (255, 0, 0), (0, 0, 255), (255, 0, 0), (0, 0, 255),
        (255, 0, 0), (0, 0, 255), (255, 0, 0), (0, 0, 255),
        (255, 0, 0), (0, 0, 255)]
edges = [[0, 1], [0, 2], [1, 3], [2, 4], 
                    [3, 5], [4, 6], [5, 6], 
                    [5, 7], [7, 9], [6, 8], [8, 10], 
                    [5, 11], [6, 12], [11, 12], 
                    [11, 13], [13, 15], [12, 14], [14, 16]]
ec = [(255, 0, 0), (0, 0, 255), (255, 0, 0), (0, 0, 255), 
                 (255, 0, 0), (0, 0, 255), (255, 0, 255),
                 (255, 0, 0), (255, 0, 0), (0, 0, 255), (0, 0, 255),
                 (255, 0, 0), (0, 0, 255), (255, 0, 255),
                 (255, 0, 0), (255, 0, 0), (0, 0, 255), (0, 0, 255)]

def write_pose(points, img): 
    points = np.array(points, dtype=np.int32).reshape(17, 2)
    for j in range(17):
        cv2.circle(img,(points[j, 0], points[j, 1]), 3, colors_hp[j], -1)
    for j, e in enumerate(edges):
          if points[e].min() > 0:
            #ec[j]で色を決めている( 左右で分ける)
            cv2.line(img, (points[e[0], 0], points[e[0], 1]),(points[e[1], 0], points[e[1], 1]), ec[j], 2,lineType=cv2.LINE_AA)

def write_rect(img,box, cl):
    x1, y1, x2, y2 = [int(x) for x in box[:4]]
    cv2.rectangle(img, (x1,y1), (x2, y2), (255, 255, 255),3)

In [12]:
VIDEO_PATH = "/content/drive/MyDrive/human_walking.mp4"

In [23]:
cap = cv2.VideoCapture(VIDEO_PATH)
frame_count = round(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #総フレーム数
fps = round(cap.get(cv2.CAP_PROP_FPS)) #fps

height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #画像高さ
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #画像幅
size = (height, width) #画像サイズ

#解析データの保存先
out_path = VIDEO_PATH[:-4] + '_result1.mp4'
out = cv2.VideoWriter(
        out_path, 
        cv2.VideoWriter_fourcc('m','p','4', 'v'),
        fps,
        size
        )

for _ in tqdm(range(frame_count)):
    ret0, frame_read = cap.read()
    if not ret0:
        break
    frame_read = cv2.rotate(frame_read, cv2.ROTATE_90_CLOCKWISE)    
    ret = detector.run(frame_read)['results']
    
    for bbox in ret[1]:
        if bbox[4] > 0.5:
            points = np.array(bbox[5:39], dtype=np.int32).reshape(17, 2)
            write_rect(frame_read, bbox[:4], 0)
            write_pose(points,frame_read)
    
    out.write(frame_read)

cap.release()
out.release()

  0%|          | 0/901 [00:00<?, ?it/s]